In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%time

df = pd.read_csv("/kaggle/input/titanic/train.csv")


In [ ]:
sex_bin = {"female": 0,
           "male": 1}
df['Sex'] = df['Sex'].map(sex_bin)


In [ ]:
import re 
def get_title(name):
    first = name.split(',')
    second = first[1].split('.')
    return second[0]


In [ ]:
df.Name = df.Name.apply(get_title)

We REPLACED NAME TO TITLE:
* 17 UNIQUE TITLES
* TITLES ARE HELPFUL FOR PREDICTING AGE 
* TITLE HELPS TO PREDICT OTHER MISSING VALUES
* NOW I AM ANALYSING TITLES AND OTHER FEATURES


In [ ]:
def best_age(name):
    return df.loc[(df.Name == name)].Age.mean()

WHAT IS DIFFERENCE BETWEEN MEAN & MEDIAN ?

In [ ]:
df.Age=df.Age.fillna(df.Name.apply(best_age))

In [ ]:
y_train = df.Survived
X_train = df.drop("Survived", axis=1)


In [ ]:
X_train = X_train.drop(["Ticket", "Cabin"], axis=1)

In [ ]:
X_train = X_train.drop(["PassengerId"], axis=1)

In [ ]:
X_train.head()

In [ ]:
X_train.Embarked.fillna("S", inplace=True)

In [ ]:
features = ['Pclass', 'Sex', 'Age' ,'SibSp', 'Parch', 'Fare', 'Embarked']
X = pd.get_dummies(X_train[features], drop_first=True)

In [ ]:
X.head()

In [ ]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
my_test = test.drop(["PassengerId","Cabin", "Ticket"],axis=1)

In [ ]:
test.info()

In [ ]:
my_test.Name = my_test.Name.apply(get_title)

In [ ]:
my_test.Age=my_test.Age.fillna(my_test.Name.apply(best_age))

In [ ]:
my_test.Fare.fillna(test.Fare.mean(), inplace=True)

In [ ]:
sex_bin = {"female": 0,
           "male": 1}
my_test['Sex'] = my_test['Sex'].map(sex_bin)

In [ ]:
my_test2 = my_test.drop("Name", axis=1)

In [ ]:
X_test = pd.get_dummies(my_test2, drop_first=True)

In [ ]:
X_test.head()

In [ ]:
X.isnull().sum()

In [ ]:
from sklearn.ensemble import RandomForestClassifier 


In [ ]:
model = RandomForestClassifier()


In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
params = {"n_estimators": np.arange(50,700,50),
         "max_depth": np.arange(1,10)}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)


In [ ]:
grid = GridSearchCV(estimator = model,
                   param_grid = params,
                   cv=skf,
                   n_jobs=-1,
                   verbose=1)
grid.fit(X, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_

In [ ]:
preds = grid.predict(X_test)

In [ ]:
output = pd.DataFrame({"PassengerId": test.PassengerId,
                      "Survived": preds})
output.to_csv('My_random.csv', index=False)